<a href="https://colab.research.google.com/github/yasserrida/active-learning-based-on-volumes/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from strategies import RandomSampling, LeastConfidence, MarginSampling, EntropySampling, KMeansSampling
from dataset import get_dataset, get_handler
from model import get_net
from torchvision import transforms
import numpy as np
import torch
import matplotlib.pyplot as plt